# Linear Regression - medical insurance

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import json

In [17]:
df = pd.read_csv('medical_insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [18]:
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [20]:
df['sex']

0       female
1         male
2         male
3         male
4         male
         ...  
1333      male
1334    female
1335    female
1336    female
1337    female
Name: sex, Length: 1338, dtype: object

In [21]:
df['sex'].replace({'female':0,'male':1},inplace = True)

In [22]:
df['smoker'].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [23]:
df['smoker'].replace({'no':1,'yes':0},inplace = True)

In [24]:
df['region'].value_counts().to_dict()

{'southeast': 364, 'southwest': 325, 'northwest': 325, 'northeast': 324}

In [25]:
df = pd.get_dummies(df ,columns=['region'])
df

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,0,16884.92400,0,0,0,1
1,18,1,33.770,1,1,1725.55230,0,0,1,0
2,28,1,33.000,3,1,4449.46200,0,0,1,0
3,33,1,22.705,0,1,21984.47061,0,1,0,0
4,32,1,28.880,0,1,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,1,10600.54830,0,1,0,0
1334,18,0,31.920,0,1,2205.98080,1,0,0,0
1335,18,0,36.850,0,1,1629.83350,0,0,1,0
1336,21,0,25.800,0,1,2007.94500,0,0,0,1


In [26]:
df['children'].value_counts()

0    574
1    324
2    240
3    157
4     25
5     18
Name: children, dtype: int64

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   uint8  
 7   region_northwest  1338 non-null   uint8  
 8   region_southeast  1338 non-null   uint8  
 9   region_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


## Train Test split

In [29]:
df = df.select_dtypes(exclude=object) 
x = df.drop('charges',axis = 1)
y = df['charges']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=34)

In [30]:
x_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
414,19,0,35.150,0,1,0,1,0,0
1279,25,0,26.790,2,1,0,1,0,0
647,40,0,23.370,3,1,1,0,0,0
764,45,0,25.175,2,1,1,0,0,0
1133,52,0,18.335,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
453,20,1,29.735,0,1,0,1,0,0
324,29,1,27.200,0,1,0,0,0,1
1109,45,1,20.350,3,1,0,0,1,0
490,19,0,32.900,0,1,0,0,0,1


# Model Training

In [31]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [37]:
# Testing Data Evaluation
y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_test, y_pred)
print("MAE :",mae)

r2 = r2_score(y_test, y_pred)
print('R-Squared :',r2)

MSE : 41587655.82548302
RMSE : 6448.849186132594
MAE : 4420.543383291357
R-Squared : 0.7442111491697807


In [38]:
# Training Data Evaluation

y_pred_train = model.predict(x_train)
mse = mean_squared_error(y_train, y_pred_train)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_train, y_pred_train)
print("MAE :",mae)

r2 = r2_score(y_train, y_pred_train)
print('R-Squared :',r2)

MSE : 35291933.482411794
RMSE : 5940.701430168982
MAE : 4101.112694905446
R-Squared : 0.7519747760752101


# Testing on Single Row

In [18]:
# x.head(1).T

In [57]:
age = 45
sex = 'female'
bmi = 30.7
children = 3
smoker = 'yes'
region = 'southeast'

# charges = ?

In [45]:
label_encoded_data = {"sex" : {'female':0,'male':1},
                     "smoker" : {'no':1,'yes':0},
                     'columns' : list(x.columns)}
label_encoded_data 

{'sex': {'female': 0, 'male': 1},
 'smoker': {'no': 1, 'yes': 0},
 'columns': ['age',
  'sex',
  'bmi',
  'children',
  'smoker',
  'region_northeast',
  'region_northwest',
  'region_southeast',
  'region_southwest']}

In [42]:
project_data = {"sex" : {'female':0,'male':1},
                     "smoker" : {'no':1,'yes':0},
                     'columns' : list(x.columns)}


In [49]:
column_names = x.columns
column_names

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region_northeast',
       'region_northwest', 'region_southeast', 'region_southwest'],
      dtype='object')

In [50]:
region =  'region_' + region
region_index = np.where(column_names == region)[0][0]

In [58]:
test_array = np.zeros(x.shape[1])
test_array[0] = age
test_array[1] = label_encoded_data['sex'][sex]
test_array[2] = bmi
test_array[3] = children
test_array[4] = label_encoded_data['smoker'][smoker]
test_array[region_index] = 1

test_array

array([45. ,  0. , 30.7,  3. ,  0. ,  0. ,  0. ,  1. ,  0. ])

In [59]:
model.predict([test_array])

C:\Users\Atishay\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([33974.0903988])

In [60]:
with open('Linear_Model.pkl','wb') as f:
    pickle.dump(model,f)

In [61]:
with open("project_data.json",'w') as f:
    json.dump(project_data,f)